# EnergyID Webhook V2 Demo

This notebook demonstrates the new EnergyID Webhook V2 API, which includes device provisioning, claiming, and the new data format for sending measurements to EnergyID.

The new webhook implementation offers several advantages:

- Simplified device registration through a claiming process
- Token-based authentication with automatic refresh
- Standardized metric types for common energy measurements
- Support for batch uploads and data aggregation
- More efficient data format for transmitting measurements

## Summary

This notebook demonstrates the EnergyID Webhook V2 API with the following features:

1. **Device Provisioning**: Setting up a device with a unique identifier
2. **Claiming Process**: Allowing users to claim a device and link it to their EnergyID account
3. **Token-based Authentication**: Automatic token refresh and handling of expired tokens
4. **Simplified Data Format**: Using standardized keys like `el`, `pv`, `gas` for common energy metrics
5. **Batch Data Uploads**: Sending multiple metrics in a single request
6. **Custom Timestamps**: Attaching specific timestamps to data
7. **Prefixed Metrics**: Using prefixes to handle multiple metrics of the same type
8. **Sensor Objects**: Managing sensor state and synchronization
9. **Automatic Synchronization**: Periodically sending updates without manual intervention

The new Webhook V2 API provides a more efficient and user-friendly way to integrate devices with EnergyID.

## 1. Setup & Dependencies

First, let's install the required dependencies if needed and import the necessary libraries:

In [ ]:
import asyncio
import datetime as dt
import json
import logging
import random
import uuid
import os

from dotenv import load_dotenv

# Configure logging for better output
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("energyid-demo")
# Load environment variables if not already loaded
load_dotenv()

## credentials
We load in credentials from the environment, or the environment file.
Refer to the example file [`.env.example`](../.env.example) for more details on the required credentials. 

In [ ]:
credentials = {
    "PROVISIONING_KEY": os.getenv("PROVISIONING_KEY"),
    "PROVISIONING_SECRET": os.getenv("PROVISIONING_SECRET"),
    "ENERGYID_DEVICE_ID": os.getenv("ENERGYID_DEVICE_ID"),
    "ENERGYID_DEVICE_NAME": os.getenv("ENERGYID_DEVICE_NAME"),
}
credentials

## 2. WebhookClient Implementation

we'll import our implementation from `energyid_webhooks.client` module. Let's look at how to use it:

In [ ]:
# Import the WebhookClient module
from energyid_webhooks.client_v2 import WebhookClient

In [ ]:
%pwd

## 3. Device Provisioning and Claiming

The first step in using the EnergyID Webhook V2 API is device provisioning. This process involves:

1. Creating a unique device identifier
2. Authenticating with EnergyID using provisioning credentials
3. Claiming the device through the EnergyID web interface

Let's set up the client with credentials from environment variables:

NOTE: you can also use existing device by specifying it in the .env. To not have to add a new device.

In [ ]:
# Get the saved credentials
provisioning_key = credentials["PROVISIONING_KEY"]
provisioning_secret = credentials["PROVISIONING_SECRET"]
device_id = credentials.get("ENERGYID_DEVICE_ID")
device_name = credentials.get("ENERGYID_DEVICE_NAME")

if not provisioning_key or not provisioning_secret:
    raise ValueError("Please set CLIENT_ID and CLIENT_SECRET environment variables")


print("Provisioning Key:", provisioning_key)
print("Provisioning Secret:", provisioning_secret)

Credentials gotten, now create unique ID for webhook device *or if already exists*

In [ ]:
if not device_name:
    device_name = "Jupyter Demo Device"
if not device_id:
    device_id = f"jupyter_demo_{uuid.uuid4().hex[:8]}"
    print(f"Generated new device ID: {device_id}")
else:
    print(f"Using existing device ID from environment: {device_id}")

Instantiate the client

In [ ]:
client = WebhookClient(
    provisioning_key=provisioning_key,
    provisioning_secret=provisioning_secret,
    device_id=device_id,
    device_name=device_name,
    firmware_version="1.0.0",
)
# Display device info for future reference
print("\nDevice info for future reference:")
print(f"device_id: {client.device_id}")
print(f"device_name: {client.device_name}")
print(f"firmware_version: {client.firmware_version}")
print("To reuse this device in future runs, set these environment variables:")
print(f"ENERGYID_DEVICE_ID={client.device_id}")
print(f"ENERGYID_DEVICE_NAME={client.device_name}")

Now let's authenticate and check if the device is already claimed:

In [ ]:
# Authenticate with EnergyID
is_claimed = await client.authenticate()

if is_claimed:
    print("✅ Device is already claimed and ready to send data!")
    print(f"Webhook URL: {client.webhook_url}")
    print(f"Auth valid until: {client.auth_valid_until}")
    print(f"\nWebhook policy: {json.dumps(client.webhook_policy, indent=2)}")
else:
    # Device needs to be claimed
    claim_info = client.get_claim_info()
    print("⚠️ Device needs to be claimed before sending data!")
    print(f"\nClaim Code: {claim_info['claim_code']}")
    print(f"Claim URL: {claim_info['claim_url']}")
    print(f"Valid until: {claim_info['valid_until']}")
    print("\n1. Visit the claim URL above in your browser")
    print("2. Log in to your EnergyID account if needed")
    print("3. Enter the claim code shown above")
    print("4. Once claimed, re-run this cell to continue or run the next cell")

If the device isn't claimed yet, follow the instructions above to claim it through the EnergyID web interface. Then re-run the cell or the next one to verify it's been claimed successfully.

In [ ]:
# Authenticate with EnergyID
is_claimed = await client.authenticate()

if is_claimed:
    print("✅ Device is already claimed and ready to send data!")
    print(f"Webhook URL: {client.webhook_url}")
    # print(f"client info: {client.client_info}")
    print(f"Auth valid until: {client.auth_valid_until}")
    print(f"\nWebhook policy: {json.dumps(client.webhook_policy, indent=2)}")
else:
    raise ValueError("Device is not claimed yet, please claim it first")

In [ ]:
client.get_claim_info()

## 4. Sending Data in the New Format

Once the device is claimed, we can start sending data. The new V2 API uses a simpler key-value structure with standardized metric keys:

In [ ]:
# Check if device is claimed before proceeding
if not client.is_claimed:
    print("⚠️ Please claim the device first!")
else:
    # Create a simple data point with current timestamp
    data = {
        # ts is automatically added if not provided
        "el": 1250.5,  # Electricity consumption in kWh
        "pv": 3560.2,  # Solar production in kWh
    }

    # Send the data
    await client.send_data(data)
    print("✅ Data sent successfully!")

## 5. Sending Data with Specific Timestamp

We can also send data with a specific timestamp instead of using the current time:

In [ ]:
# Check if device is claimed before proceeding
if not client.is_claimed:
    print("⚠️ Please claim the device first!")
else:
    # Create data for a specific time (yesterday)
    yesterday = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=1)
    metrics = {"el": 1240.3, "pv": 3550.1}

    # Send with specific timestamp
    await client.send_data(metrics, timestamp=yesterday)
    print(f"✅ Data sent with timestamp {yesterday.isoformat()}")

## 7. Using Prefixed Metrics

The new API also supports prefixed metrics to handle multiple data points of the same type:

In [ ]:
# Check if device is claimed before proceeding
if not client.is_claimed:
    print("⚠️ Please claim the device first!")
else:
    # Create data with prefixed metrics
    metrics = {
        # Electricity with time-of-use tariffs
        "el.t1": 850.5,  # Day tariff
        "el.t2": 410.2,  # Night tariff
        # Electricity injection with time-of-use tariffs
        "el-i.t1": 120.3,  # Day tariff
        "el-i.t2": 35.7,  # Night tariff
        # Custom sensors
        "temperature.living": 21.5,
        "temperature.outside": 15.2,
    }

    # Send with prefixed metrics
    await client.send_data(metrics)
    print("✅ Prefixed metrics sent successfully!")

In [ ]:
# we can refresh the client to make sure we have correct webhook policy, webhook url headers
await client.refresh_webhook()

## 8. Using Sensor Objects

The WebhookClient provides a Sensor class to manage each sensor's state and synchronization:

In [ ]:
# Check if device is claimed before proceeding
if not client.is_claimed:
    print("⚠️ Please claim the device first!")
else:
    # Set up some sensors and update their values
    await client.update_sensor("el", 1260.0)
    await client.update_sensor("el-i", 155.2)
    await client.update_sensor("pv", 3580.0)
    await client.update_sensor("gas", 455.3)
    print("✅ Sensors updated successfully!")

    # Synchronize the updated sensors
    await client.synchronize_sensors()
    print("✅ Sensors synchronized successfully!")

## 9. Simulating Continuous Data Updates

Let's simulate how you might continuously update data over time:

In [ ]:
async def simulate_updates(client, updates=5, interval=2):
    """Simulate sensor updates over time."""
    if not client.is_claimed:
        print("⚠️ Please claim the device first!")
        return

    el_value = 1260.0  # Initial electricity consumption
    pv_value = 3580.0  # Initial solar production
    temp_value = 460  # Initial living room temperature

    # Set up sensors if they don't exist
    await client.update_sensor("el", el_value)
    await client.update_sensor("pv", pv_value)
    await client.update_sensor("gas", temp_value)

    print(f"Simulating {updates} updates at {interval} second intervals...")

    for i in range(updates):
        # Update values with small random changes
        el_value += random.uniform(0.1, 0.5)  # Small increase in consumption
        pv_value += random.uniform(0.2, 1.0)  # Larger increase in production
        temp_value = max(18, min(25, temp_value + random.uniform(-0.5, 0.5)))

        # Update sensors
        await client.update_sensor("el", el_value)
        await client.update_sensor("pv", pv_value)
        await client.update_sensor("gas", temp_value)

        now = dt.datetime.now()
        print(
            f"[{now.isoformat()}] Update {i + 1}/{updates}: el={el_value:.2f}, "
            f"pv={pv_value:.2f}, temp={temp_value:.1f}"
        )

        # Synchronize after each update (in a real application, you'd do this less frequently)
        if i % 2 == 1:  # Sync every other update
            print("   Synchronizing...")
            await client.synchronize_sensors()

        # Wait between updates
        if i < updates - 1:  # Don't wait after the last update
            await asyncio.sleep(interval)

    print("\n✅ Simulation complete!")


# Run the simulation
await simulate_updates(client, updates=100, interval=60)

## 10. Auto-synchronization

The WebhookClient supports automatic synchronization at a specified interval:

In [ ]:
# Start auto-sync with a 30-second interval
client.start_auto_sync(30)
print("✅ Auto-sync started with 30-second interval")
print("   Now you can update sensors and they will be synchronized automatically")

Let's update some sensors and rely on auto-sync to send the data:

In [ ]:
# Update sensors without manually synchronizing
await client.update_sensor("el", 1265.5)
await client.update_sensor("pv", 3590.2)
await client.update_sensor("temperature.living", 22.0)

print("✅ Sensors updated, they will be synchronized automatically")
print("   Wait for the auto-sync interval to see the data being sent")

## 11. Clean Up

Finally, let's properly close the client to clean up resources:

In [ ]:
# Close the client
await client.close()
print("✅ Client closed and resources cleaned up!")